In [1]:
import pandas as pd
#Used for data manipulation and analysis

In [2]:
df=pd.read_excel('../data/Constraint_Train.xlsx')
#Read an Excel file into a pandas DataFrame

In [3]:
df.head()
#Prints top 5 rows of excel file

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [4]:
df=df.dropna()
#Drops the rows containing Null values

In [5]:
X=df.drop('label',axis=1)
#Drops the independent features, and stores it into X

In [6]:
y=df['label']
#Drops the dependent features, and stores it into X

In [7]:
X.shape
#This returns the number of rows and columns as a tuple

(6420, 2)

In [8]:
y.shape
#This returns the number of rows as a tuple

(6420,)

In [9]:
import tensorflow as tf
#It can be used across a range of tasks but has a particular focus
# on training and inference of deep neural networks

In [10]:
from tensorflow.keras.layers import Embedding
#embedding is a dense vector of floating point values
# (the length of the vector is a parameter that is specified).

from tensorflow.keras.preprocessing.sequence import pad_sequences
#To make each input length fixed

from tensorflow.keras.models import Sequential
#A Sequential model is appropriate for a plain stack of layers 
# where each layer has exactly one input tensor and one output tensor.

from tensorflow.keras.preprocessing.text import one_hot
#One-hot encodes a text into a list of word indexes of size n.
# It returns a list of encoded integers each corresponding to a word 
# (or token) in the given input string.

from tensorflow.keras.layers import LSTM
#Imports LSTM Classificatio model from Keras

from tensorflow.keras.layers import Dense
#Dense implements the operation:
# output = activation(dot(input, kernel) + bias). 
# These are all attributes of Dense. 

In [11]:
voc_size=5000
# Specify the vocabulary size, to be used later.

In [12]:
messages=X.copy()
#Copies the tuple X into variable messages

In [13]:
messages.reset_index(inplace=True)
#Reset the index, or a level of messages dataframe 
# to get a new order of arrangement.

In [14]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
#It is a suite of libraries and programs for symbolic and statistical natural language 
# processing (NLP) for English written in the Python programming language. 

import re
#A regular expression (or RE) specifies a set of strings that matches it. The functions 
# in this module is to check if a particular string matches a given regular expression.

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
#Dataset Preprocessing is done here
from nltk.stem.porter import PorterStemmer
#Algorithm to for removing the commoner morphological 
# and inflexional endings from words in English. 
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['tweet'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
#One-hot encodes a text into a list of word indexes of size n.
# It returns a list of encoded integers each corresponding to a word 
# (or token) in the given input string.

In [17]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0 4604 ... 1414 1299 4698]
 [   0    0    0 ...  885 1139 4439]
 [   0    0    0 ... 1123 2074 1585]
 ...
 [   0    0    0 ... 2198 1766 1540]
 [   0    0    0 ... 4733 1851  994]
 [2510 4072 1527 ... 3011 3721 4533]]


In [18]:
#Creating a model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())
###### COmment again from here bhiru

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
len(embedded_docs),y.shape

(6420, (6420,))

In [20]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [21]:
X_final.shape,y_final.shape

((6420, 20), (6420,))

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [23]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "D:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "D:\Anaconda\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "D:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "D:\Anaconda\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
      app.start()
    File "D:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
      self.io_loop.start()
    File "D:\Anaconda\lib\site-packages\tornado\platform\asyncio.py", line 153, in start
      self.asyncio_loop.run_forever()
    File "D:\Anaconda\lib\asyncio\base_events.py", line 538, in run_forever
      self._run_once()
    File "D:\Anaconda\lib\asyncio\base_events.py", line 1782, in _run_once
      handle._run()
    File "D:\Anacon